In [1]:
### Project 2 Airbnb Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
import glob

import plotly
import plotly.plotly as py
py.sign_in('nzahadat', 'VDBLh2LQZvNWxovGAbr8')
print(plotly.__version__)
plotly.offline.init_notebook_mode() # run at the start of every notebook


%matplotlib inline

2.0.11


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
# import & conbine data
total_listings=pd.DataFrame()
for filename in glob.iglob('./**/**/listings.csv',recursive=False):
    print(filename)
    data=pd.read_csv(filename,error_bad_lines=False,low_memory=False,encoding='ISO-8859-1')
    total_listings=total_listings.append(data)
    
    

.\2015-01-01\data\listings.csv
.\2015-03-01\data\listings.csv
.\2015-04-01\data\listings.csv
.\2015-05-01\data\listings.csv
.\2015-06-01\data\listings.csv
.\2015-08-01\data\listings.csv
.\2015-09-01\data\listings.csv
.\2015-10-01\data\listings.csv
.\2015-11-01\data\listings.csv
.\2015-11-20\data\listings.csv
.\2015-12-02\data\listings.csv
.\2016-01-01\data\listings.csv
.\2016-02-02\data\listings.csv
.\2016-04-03\data\listings.csv
.\2016-05-02\data\listings.csv
.\2016-06-02\data\listings.csv
.\2016-07-02\data\listings.csv
.\2016-08-02\data\listings.csv
.\2016-09-02\data\listings.csv
.\2016-10-01\data\listings.csv
.\2016-11-02\data\listings.csv
.\2016-12-03\data\listings.csv
.\2017-01-01\data\listings.csv
.\2017-02-02\data\listings.csv
.\2017-03-02\data\listings.csv
.\2017-04-02\data\listings.csv
.\2017-05-02\data\listings.csv


In [57]:
# create row index as id + calendar_last_scraped
total_listings['id_date']=total_listings['id'].map(str)+"_"+total_listings['calendar_last_scraped']
total_listings=total_listings.set_index(['id_date'])

In [47]:
total_listings.shape

(941484, 97)

In [48]:
total_listings.isnull().sum()/total_listings.shape[0]*100

access                              70.335555
accommodates                         0.000000
amenities                           14.518781
availability_30                      0.000319
availability_365                     0.000319
availability_60                      0.000319
availability_90                      0.000319
bathrooms                            0.756147
bed_type                             0.000000
bedrooms                             0.256191
beds                                 0.225283
calculated_host_listings_count       2.909449
calendar_last_scraped                0.000000
calendar_updated                     0.000000
cancellation_policy                 14.518781
city                                 0.082954
cleaning_fee                        41.541226
country                              0.000106
country_code                        14.518781
description                          2.937172
experiences_offered                 14.518781
extra_people                      

In [58]:
# drop columns which have missing value over 15% and columns that are not related to analysis
total_listings_clean=total_listings.dropna(thresh=0.85*total_listings.shape[0],axis=1)
total_listings_clean=total_listings_clean.drop(['calendar_updated','country_code','country','experiences_offered',
                                                'host_total_listings_count','market','neighbourhood','neighbourhood_group_cleansed',
                                                'scrape_id','smart_location','street','summary','host_picture_url',
                                               'host_thumbnail_url','host_url','listing_url','picture_url'],axis=1)

In [59]:
list(total_listings_clean)

['accommodates',
 'amenities',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bed_type',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'calendar_last_scraped',
 'cancellation_policy',
 'city',
 'description',
 'extra_people',
 'guests_included',
 'host_has_profile_pic',
 'host_id',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_location',
 'host_name',
 'host_since',
 'host_verifications',
 'id',
 'instant_bookable',
 'is_location_exact',
 'last_scraped',
 'latitude',
 'longitude',
 'maximum_nights',
 'minimum_nights',
 'name',
 'neighbourhood_cleansed',
 'number_of_reviews',
 'price',
 'property_type',
 'require_guest_phone_verification',
 'require_guest_profile_picture',
 'requires_license',
 'room_type',
 'state',
 'zipcode']

In [60]:
total_listings_clean.head()


,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,...,neighbourhood_cleansed,number_of_reviews,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,room_type,state,zipcode
id_date,,,,,,,,,,,,,,,,,,,,,
1069266_2015-01-02,2,NaN,21.0,322.0,51.0,72.0,1.0,Real Bed,1.0,1.0,...,Midtown East,62,$160.00,Apartment,NaN,NaN,NaN,Entire home/apt,NY,10022
1846722_2015-01-02,10,NaN,28.0,348.0,58.0,88.0,1.0,Real Bed,3.0,3.0,...,Hamilton Heights,22,$105.00,Apartment,NaN,NaN,NaN,Entire home/apt,NY,NaN
2061725_2015-01-02,2,NaN,4.0,227.0,13.0,26.0,1.0,Real Bed,1.0,2.0,...,Bushwick,35,$58.00,Apartment,NaN,NaN,NaN,Private room,NY,11221
44974_2015-01-03,2,NaN,1.0,274.0,1.0,1.0,1.0,Real Bed,1.0,1.0,...,West Village,26,$185.00,Apartment,NaN,NaN,NaN,Entire home/apt,NY,10011
4701675_2015-01-02,2,NaN,30.0,365.0,60.0,90.0,1.0,Real Bed,1.0,2.0,...,Chelsea,1,$195.00,Apartment,NaN,NaN,NaN,Entire home/apt,NY,10011


In [61]:
# parse amenities
total_listings_clean.amenities = total_listings_clean.amenities.str.replace("[{}]", "")
amenities = total_listings_clean.amenities.str.get_dummies(sep = ",")


In [52]:
amenities.shape


(941484, 122)

In [62]:
amenities


,"""24-Hour Check-in""","""24-hour check-in""","""Accessible-height bed""","""Accessible-height toilet""","""Air Conditioning""","""Air conditioning""","""BBQ grill""","""Baby bath""","""Baby monitor""","""Babysitter recommendations""",...,Lockbox,Microwave,Oven,Pool,Refrigerator,Shampoo,Smartlock,Stove,TV,Washer
id_date,,,,,,,,,,,,,,,,,,,,,
1069266_2015-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1846722_2015-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2061725_2015-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44974_2015-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4701675_2015-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68914_2015-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4832596_2015-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2562510_2015-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3005360_2015-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# merge edited amenities back to total_listings
total_listings_clean_ame=total_listings_clean.merge(amenities,left_index=True,right_index=True)
total_listings_clean_ame

,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,...,Lockbox,Microwave,Oven,Pool,Refrigerator,Shampoo,Smartlock,Stove,TV,Washer
id_date,,,,,,,,,,,,,,,,,,,,,
1069266_2015-01-02,2,NaN,21.0,322.0,51.0,72.0,1.0,Real Bed,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1846722_2015-01-02,10,NaN,28.0,348.0,58.0,88.0,1.0,Real Bed,3.0,3.0,...,0,0,0,0,0,0,0,0,0,0
2061725_2015-01-02,2,NaN,4.0,227.0,13.0,26.0,1.0,Real Bed,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
44974_2015-01-03,2,NaN,1.0,274.0,1.0,1.0,1.0,Real Bed,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4701675_2015-01-02,2,NaN,30.0,365.0,60.0,90.0,1.0,Real Bed,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
68914_2015-01-02,6,NaN,11.0,287.0,33.0,57.0,1.0,Real Bed,2.0,3.0,...,0,0,0,0,0,0,0,0,0,0
4832596_2015-01-03,2,NaN,29.0,364.0,59.0,89.0,1.0,Real Bed,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2562510_2015-01-02,2,NaN,5.0,273.0,5.0,5.0,1.0,Real Bed,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3005360_2015-01-03,4,NaN,30.0,274.0,60.0,90.0,1.0,Real Bed,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
